In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.joplinempiricalrestoration import JoplinEmpiricalRestoration
import pyincore.globals as pyglobals

In [ ]:
client = IncoreClient()

### Start with Joplin building damage analysis

In [ ]:
# Joplin, MO building dataset
bldg_dataset_id = "5dbc8478b9219c06dd242c0d"

bldg_dmg = BuildingDamage(client)
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

mapping_id = "5d8942dfb9219c068ea795ea"  # 19 archetype, non-retrofit
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")

In [ ]:
# The simulated EF-5 tornado shows geographical locations and the range of wind speed
# of tornado hazard in Joplin.
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"

In [ ]:
# parameters
result_name = "joplin_tornado_dmg_result"
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)
bldg_dmg.set_parameter("seed", 1000)
bldg_dmg.run_analysis()
# end of Building damage analysis

In [ ]:
# get csv results from Building damage analysis
building_dmg_result = bldg_dmg.get_output_dataset("ds_result")
building_dmg_result.get_dataframe_from_csv()

### Start with Joplin restoration model

The example inventory is for Joplin, MO, after 2011 tornado.

In [ ]:
restoration = JoplinEmpiricalRestoration(client)

In [ ]:
# Joplin, MO building damage
restoration.load_remote_input_dataset("buildings", bldg_dataset_id)
# restoration.load_remote_input_dataset("building_dmg", building_dmg_result)
restoration.set_input_dataset("building_dmg", building_dmg_result)

In [ ]:
# parameters
result_name = "Joplin_empirical_restoration_result"
restoration.set_parameter("result_name", result_name)
restoration.set_parameter("target_functionality_level", 0)
restoration.set_parameter("seed", 1234)

In [ ]:
# Target functionality level is set to FL0, full restoration in the example above. A user has an option 
# to set target functionality level for each building by creating a csv file with two columns; 
# guid of a building and target level from (integer) 0 to 3. The file must be uploaded to IN-CORE data service.
# The dataset id is then used in the notebook:
#
# restoration.load_remote_input_dataset("building_functionality_level", building_fl_id)

In [ ]:
# Run analysis and show results
restoration.run_analysis()

restoration_result = restoration.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
df_bldg_restoration = restoration_result.get_dataframe_from_csv()
# Display top 5 rows of output data
df_bldg_restoration[df_bldg_restoration["restorDays"] > 0]